Idea is to try out 'pysolar' package. https://pysolar.readthedocs.io/en/latest/. This data could be used to enhance dataset. 


<img src="./angles.png" alt="drawing" width="400"/>

In [17]:
from pysolar import solar
import datetime

# roughly london cooridnates
latitude = 51
longitude = 0

# get the altitude angle right now
date = datetime.datetime.now(datetime.timezone.utc)
a = solar.get_altitude(latitude, longitude, date)
print(f'Altitude angle right now is {a}')

# get the altitude angle at 12.00 on midsummer
date = datetime.datetime(2021,6,22,12,tzinfo=datetime.timezone.utc)
a = solar.get_altitude(latitude, longitude, date)
print(f'Mid-summer altitude angle is {a}')

# get the altitude angle at 12.00 on mid winter
date = datetime.datetime(2021,12,22,12,tzinfo=datetime.timezone.utc)
a = solar.get_altitude(latitude, longitude, date)
print(f'Mid-winter altitude angle is {a}')

Altitude angle right now is 39.0512676824061
Mid-summer altitude angle is 62.43594192126586
Mid-winter altitude angle is 15.620331125576355


/Users/peterdudfield/miniconda3/lib/python3.9/site-packages/pysolar/solartime.py:110: UserWarning: I don't know about leap seconds after 2020
  warnings.warn \


In [11]:
# azimuth angle of the sun

# get the altitude angle at 12.00 on mid winter
date = datetime.datetime(2021,12,22,12,tzinfo=datetime.timezone.utc)
a = solar.get_azimuth(latitude, longitude, date)
print(f'Mid-winter azimuth angle is {a}')

# loop over hours in the day
for hour in range(0,24):
    date = datetime.datetime(2021,12,22,hour,tzinfo=datetime.timezone.utc)
    a = solar.get_azimuth(latitude, longitude, date)
    print(f'Mid-winter azimuth angle is {a} at {hour} hours')
    


Mid-winter azimuth angle is 180.33261684527582
Mid-winter azimuth angle is 0.8153803179842214 at 0 hours
Mid-winter azimuth angle is 29.145159449446993 at 1 hours
Mid-winter azimuth angle is 51.8515681739288 at 2 hours
Mid-winter azimuth angle is 69.02755191178824 at 3 hours
Mid-winter azimuth angle is 82.71328071923766 at 4 hours
Mid-winter azimuth angle is 94.53313692070014 at 5 hours
Mid-winter azimuth angle is 105.53413678796936 at 6 hours
Mid-winter azimuth angle is 116.4101074342384 at 7 hours
Mid-winter azimuth angle is 127.65326347530785 at 8 hours
Mid-winter azimuth angle is 139.60911204460584 at 9 hours
Mid-winter azimuth angle is 152.45582436419454 at 10 hours
Mid-winter azimuth angle is 166.13922539463377 at 11 hours
Mid-winter azimuth angle is 180.33261684527582 at 12 hours
Mid-winter azimuth angle is 194.5130741106809 at 13 hours
Mid-winter azimuth angle is 208.16296140182246 at 14 hours
Mid-winter azimuth angle is 220.96769616945144 at 15 hours
Mid-winter azimuth angle i

In [49]:
# Working this out for a pandas series
import pandas as pd
import numpy as np
import time

# create fake data
data = pd.DataFrame(index=pd.date_range(start='2021-01-01', end='2021-01-05', tz='UTC', freq='5T'))
data['latitude'] = np.random.uniform(-90,90,len(data))  
data['longitude'] = np.random.uniform(-180,180,len(data))
data['datestamp'] = data.index.to_pydatetime()

t = time.time()
data['altitude'] = data.apply(
      lambda row: solar.get_altitude(row.latitude, 
                                     row.longitude, 
                                     row.datestamp.to_pydatetime()), axis=1)

data['azimuth'] = data.apply(
      lambda row: solar.get_azimuth(row.latitude, 
                                     row.longitude, 
                                     row.datestamp.to_pydatetime()), axis=1)

# seems to take about 2 seconds per 1000 datapoints. Is this fast enough?
print(f'Took {time.time()-t} seconds to make {len(data)} data points')
print(data.head())



Took 2.7064759731292725 seconds to make 1153 data points
                            latitude   longitude                 datestamp  \
2021-01-01 00:00:00+00:00   0.834009  172.420297 2021-01-01 00:00:00+00:00   
2021-01-01 00:05:00+00:00  13.949034 -155.088192 2021-01-01 00:05:00+00:00   
2021-01-01 00:10:00+00:00  14.682011 -108.658959 2021-01-01 00:10:00+00:00   
2021-01-01 00:15:00+00:00 -28.723342    5.082953 2021-01-01 00:15:00+00:00   
2021-01-01 00:20:00+00:00 -62.363635  -74.302603 2021-01-01 00:20:00+00:00   

                            altitude     azimuth  
2021-01-01 00:00:00+00:00  64.802818  161.534120  
2021-01-01 00:05:00+00:00  45.526560  214.172977  
2021-01-01 00:10:00+00:00   9.381081  243.120716  
2021-01-01 00:15:00+00:00 -37.709818  170.697741  
2021-01-01 00:20:00+00:00  11.678957  242.115167  


In [65]:
from pvlib import solarposition

tz = 'UTC'
lat, lon = 0, 51
# create fake data
times = pd.date_range('2021-01-01 00:00:00', '2021-01-05', closed='left',
                      freq='5T', tz=tz)

t = time.time()
solpos = solarposition.get_solarposition(times, lat, lon)
print(solpos.head())

# seems to take about 0.03 seconds per 1000 datapoints. Is this fast enough?
print(f'Took {time.time()-t} seconds to make {len(times)} data points')

# Note that this takes constant lat and lon in. 


                           apparent_zenith      zenith  apparent_elevation  \
2021-01-01 00:00:00+00:00       126.155548  126.155548          -36.155548   
2021-01-01 00:05:00+00:00       125.059834  125.059834          -35.059834   
2021-01-01 00:10:00+00:00       123.959754  123.959754          -33.959754   
2021-01-01 00:15:00+00:00       122.855592  122.855592          -32.855592   
2021-01-01 00:20:00+00:00       121.747615  121.747615          -31.747615   

                           elevation     azimuth  equation_of_time  
2021-01-01 00:00:00+00:00 -36.155548  118.940350         -3.431294  
2021-01-01 00:05:00+00:00 -35.059834  118.509945         -3.432928  
2021-01-01 00:10:00+00:00 -33.959754  118.102241         -3.434562  
2021-01-01 00:15:00+00:00 -32.855592  117.716144         -3.436196  
2021-01-01 00:20:00+00:00 -31.747615  117.350639         -3.437830  
Took 0.03498125076293945 seconds to make 1152 data points
